In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train = pd.read_excel("타이타닉_data.xlsx", sheetname="train")
test = pd.read_excel("타이타닉_data.xlsx", sheetname="test")

In [3]:
train["train"]=1
test["train"]=0

In [4]:
data = pd.concat([train,test],0)

In [5]:
data['Title'] = data.name.str.extract(' ([A-Za-z]+)\.')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [6]:
#Mlle -> 프랑스어로 Miss
#Mme -> 마담, 부인
#Don -> 스페인어로 남성
#Dona -> 스페인어로 여성
#Dr -> doctor
#Rev -> 목사에 대한 극존칭
#Major -> 소령
#Jonkheer -> 귀족
#Capt -> 함장
#Col -> 대령
#Countess -> 귀족
data["Title"] = data["Title"].replace(['Countess', 'Dr', 'Jonkheer', 'Rev',"Lady","Master"], 'Other')
data['Title'] = data['Title'].replace('Don', 'Mr')
data['Title'] = data['Title'].replace('Dona', 'Mrs')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
#data['Title'] = data['Title'].replace('Lady', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data["Title"] = data["Title"].replace(["Major","Col","Capt" ,'Sir'], 'Navy')

In [7]:
data["family"] = data["sibsp"]+data["parch"]
data.loc[data['family'] > 0, 'travelled_alone'] = 'No'
data.loc[data['family'] == 0, 'travelled_alone'] = 'Yes'

In [8]:
data['pclass'] = data['pclass'].astype(str)

In [9]:
ages = pd.cut(data["age"], [0,18,35,46,60,100], labels=False, retbins=True, right=False)

In [10]:
data["emb_age"] = ages[0]
data['emb_age'] = data['emb_age'].astype(str)

In [11]:
data["emb"] = data["pclass"] + data["sex"] + data["travelled_alone"] + data['emb_age']

In [12]:
data.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,survival,train,Title,family,travelled_alone,emb_age,emb
0,2,"Lehmann, Miss. Bertha",Female,17,0,0,SC 1748,12.00,C,1,1,Miss,0,Yes,0,2FemaleYes0
1,3,"Laitinen, Miss. Kristina Sofia",Female,37,0,0,4135,9.59,S,0,1,Miss,0,Yes,2,3FemaleYes2
2,3,"Rosblom, Mr. Viktor Richard",Male,18,1,1,370129,20.21,S,0,1,Mr,2,No,1,3MaleNo1
3,3,"Karaic, Mr. Milan",Male,30,0,0,349246,7.90,S,0,1,Mr,0,Yes,1,3MaleYes1
4,3,"Moen, Mr. Sigurd Hansen",Male,25,0,0,348123,7.65,S,0,1,Mr,0,Yes,1,3MaleYes1


In [13]:
data["emb"].nunique()

56

In [14]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def normalize_col(data, colum):
    scaler = MinMaxScaler()
    data[colum] = scaler.fit_transform(data[colum].values.reshape(-1,1))
    return data
data= normalize_col(data,"fare")
data= normalize_col(data,"age")
#data= normalize_col(data,"family")

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
tok = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level="")

In [16]:
tok.fit_on_texts(data["emb"].values)

In [17]:
seq = tok.texts_to_sequences(data["emb"].values)

In [18]:
seq = np.array(seq)

In [19]:
data["seq"] = seq

In [20]:
data["seq"].nunique()

56

In [21]:
train_x = data[data["train"]==1][["seq", "fare", "age"]]
train_y = data[data["train"]==1]["survival"]

In [22]:
test_x = data[data["train"]==0][["seq", "fare", "age"]]
test_y = data[data["train"]==0]["survival"]

In [23]:
train_x

,seq,fare,age
0,47,0.023422,0.2125
1,40,0.018718,0.4625
2,9,0.039447,0.2250
3,1,0.015420,0.3750
4,1,0.014932,0.3125
5,1,0.015420,0.3375
6,3,0.026897,0.0125
7,14,0.182500,0.2875
8,1,0.014151,0.2750
9,13,0.025374,0.3000


In [24]:
input1 = tf.keras.layers.Input(shape=(2,), name="num")
input2 = tf.keras.layers.Input(shape=(1), name="emb")

In [25]:
emb_layer = tf.keras.layers.Embedding(57,20, embeddings_regularizer="l2")(input2)

In [26]:
#drop1 = tf.keras.layers.SpatialDropout1D(0.3)(emb_layer)

In [27]:
emb_layer = tf.keras.layers.Flatten()(emb_layer)

In [28]:
concat_layer = tf.keras.layers.concatenate([input1,emb_layer])

In [29]:
dense1 = tf.keras.layers.Dense(50,kernel_initializer="glorot_normal",use_bias=True)(concat_layer)

bat = tf.keras.layers.BatchNormalization()(dense1)

#dense2 = tf.keras.layers.Dense(50, 
#                                kernel_initializer="glorot_normal", kernel_regularizer="l2", 
#                             use_bias=True)(dense1)
re = tf.keras.layers.ReLU()(bat)
dense1 = tf.keras.layers.Dense(50,kernel_initializer="glorot_normal",use_bias=True)(re)

bat = tf.keras.layers.BatchNormalization()(dense1)

#dense2 = tf.keras.layers.Dense(50, 
#                                kernel_initializer="glorot_normal", kernel_regularizer="l2", 
#                             use_bias=True)(dense1)
re = tf.keras.layers.ReLU()(bat)

dense3 = tf.keras.layers.Dense(10, 
                                kernel_initializer="glorot_normal",
                             use_bias=True)(re)


out = tf.keras.layers.Dense(1, activation="sigmoid",
                             kernel_initializer="glorot_normal",use_bias=True)(dense3)

In [30]:
model = tf.keras.Model(inputs = [input1, input2], outputs = out)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit([train_x[["fare","age"]].values, train_x["seq"].values],train_y.values, batch_size=32,
          epochs=100, validation_data=([test_x[["fare","age"]].values, test_x["seq"].values],test_y.values))

Train on 730 samples, validate on 313 samples
Epoch 1/100


/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


730/730 [==============================] - 2s 2ms/sample - loss: 0.6674 - accuracy: 0.6479 - val_loss: 0.6959 - val_accuracy: 0.6773
Epoch 2/100
730/730 [==============================] - 0s 122us/sample - loss: 0.5058 - accuracy: 0.7685 - val_loss: 0.6921 - val_accuracy: 0.6773
Epoch 3/100
730/730 [==============================] - 0s 124us/sample - loss: 0.4539 - accuracy: 0.7863 - val_loss: 0.6934 - val_accuracy: 0.6709
Epoch 4/100
730/730 [==============================] - 0s 121us/sample - loss: 0.4327 - accuracy: 0.8123 - val_loss: 0.6968 - val_accuracy: 0.4569
Epoch 5/100
730/730 [==============================] - 0s 112us/sample - loss: 0.4334 - accuracy: 0.8123 - val_loss: 0.7007 - val_accuracy: 0.4313
Epoch 6/100
730/730 [==============================] - 0s 119us/sample - loss: 0.4173 - accuracy: 0.8178 - val_loss: 0.6988 - val_accuracy: 0.4281
Epoch 7/100
730/730 [==============================] - 0s 104us/sample - loss: 0.4212 - accuracy: 0.8205 - val_loss: 0.7050 - val_ac